# DataFrames para Funciones

In [39]:
import pandas as pd

In [40]:
data_review = pd.read_csv('../datasets/australian_reviews_listo.csv')
data_items = pd.read_csv('../datasets/australian_items.csv')
data_output= pd.read_csv('../datasets/output.csv')

In [41]:
data_review

,user_id,user_url,posted,item_id,recommend,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011,1250,True,1
1,js41637,http://steamcommunity.com/id/js41637,2014,251610,True,1
2,evcentric,http://steamcommunity.com/id/evcentric,2014,248820,True,1
3,doctr,http://steamcommunity.com/id/doctr,2013,250320,True,2
4,maplemage,http://steamcommunity.com/id/maplemage,2014,211420,True,1
...,...,...,...,...,...,...
59270,BonnieMTD,http://steamcommunity.com/id/BonnieMTD,2015,400,True,2
59271,amillionlemons,http://steamcommunity.com/id/amillionlemons,2015,313120,True,1
59272,keepit1hunid,http://steamcommunity.com/id/keepit1hunid,2014,17410,True,2
59273,SKELETRONPRIMEISOP,http://steamcommunity.com/id/SKELETRONPRIMEISOP,2014,304930,True,1


def UsersWorstDeveloper( año : int ): Devuelve el top 3 de desarrolladoras con juegos MENOS recomendados por usuarios para el año dado.   
(reviews.recommend = False y comentarios negativos)  
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [45]:
worst_reviews = data_review[(data_review['recommend'] == False) & (data_review['sentiment_analysis'] == 0)]
count_worst_reviews = worst_reviews.groupby(['item_id','posted']).size().reset_index(name='cbad_reviews')
df_users = pd.merge(count_worst_reviews,data_output)
df_users = df_users.drop(columns=['early_access','price','release_date','app_name','title','genres','publisher'])
df_users


,item_id,posted,cbad_reviews,developer
0,10,2015,1,Valve
1,20,2015,1,Valve
2,80,2014,1,Valve
3,220,2014,1,Valve
4,220,2015,1,Valve
...,...,...,...,...
1737,501760,2014,1,Kiddy
1738,501760,2014,1,Kiddy
1739,502550,2014,1,Freakinware Limited
1740,502550,2014,1,Freakinware Limited
